# Bunny ICP

In the lecture, we learned about the Iterative Closest Point (ICP) algorithm. In this exercise, you will implement the ICP algorithm to solve the standard Stanford Bunny problem!

**Learning Objectives**
- Understand and Implement the ICP algorithm

**What You'll Implement**
- The ```least_squares_transform``` function to optimize transformation given correspondence
- The ```icp``` algorithm.


## Setup and Imports

Let us first import our standard drake functionality

In [1]:
import numpy as np
from pydrake.all import PointCloud, Rgba, RigidTransform, RotationMatrix, StartMeshcat
from scipy.spatial import KDTree

from manipulation import FindResource
from manipulation.exercises.grader import Grader
from manipulation.exercises.pose.test_icp import TestICP

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


Let's first visualize the point clouds of Stanford bunny in meshcat!

In [3]:
# Visualize Stanford Bunny
xyzs = np.load(FindResource("models/bunny/bunny.npy"))
cloud = PointCloud(xyzs.shape[1])
cloud.mutable_xyzs()[:] = xyzs

# Pose for the blue bunny
X_blue = RigidTransform(RotationMatrix.MakeXRotation(np.pi / 6), [-0.1, 0.1, 0.1])

pointcloud_model = xyzs
pointcloud_scene = X_blue.multiply(xyzs)

meshcat.Delete()
meshcat.SetProperty("/Background", "visible", False)
meshcat.SetProperty("/Cameras/default/rotated/<object>", "zoom", 10.5)
meshcat.SetObject("red_bunny", cloud, point_size=0.01, rgba=Rgba(1.0, 0, 0))
meshcat.SetTransform("red_bunny", RigidTransform())
meshcat.SetObject("blue_bunny", cloud, point_size=0.01, rgba=Rgba(0, 0, 1.0))
meshcat.SetTransform("blue_bunny", X_blue)

## Part 1: Point cloud registration with known correspondences

In this section, you will follow the [derivation](http://manipulation.csail.mit.edu/pose.html#registration) to solve the optimization problem below.

$$\begin{aligned} \min_{p\in\mathbb{R}^3,R\in\mathbb{R}^{3\times3}} \quad & \sum_{i=1}^{N_s} \| p + R \hspace{.1cm} {^Op^{m_{c_i}}} - p^{s_i}\|^2, \\ s.t. \quad & RR^T = I, \quad \det(R)=1\end{aligned}$$

The goal is to find the transform that registers the point clouds of the model and the scene, assuming the correspondence is known.  You may refer to the implementation from [deepnote](https://github.com/RussTedrake/manipulation/blob/master/book/pose/icp.ipynb) and the explanation from [textbook](http://manipulation.csail.mit.edu/pose.html#icp).

**YOUR TASK**: In the cell below, implement the ```least_squares_transform``` nethod.

In [9]:
def least_squares_transform(scene: np.ndarray, model: np.ndarray) -> RigidTransform:
    """
    Calculates the least-squares best-fit transform that maps corresponding
    points scene to model.
    Args:
      scene: 3xN numpy array of corresponding points
      model: 3xM numpy array of corresponding points
    Returns:
      X_BA: A RigidTransform object that maps point_cloud_A on to point_cloud_B
            such that
                        X_BA.multiply(model) ~= scene,
    """
    def to_3xN(P):
        return P.T if P.shape[0] != 3 and P.shape[1] == 3 else P

    scene = to_3xN(scene).astype(float)
    model = to_3xN(model).astype(float)
    assert scene.shape[0] == 3 and model.shape[0] == 3, "Points must be 3xN."
    assert scene.shape[1] == model.shape[1] and scene.shape[1] >= 3, "Need N>=3 matches."

    # Centroids
    c_s = np.mean(scene, axis=1, keepdims=True)   # 3x1
    c_m = np.mean(model, axis=1, keepdims=True)   # 3x1

    # Demean
    S0 = scene - c_s
    M0 = model - c_m

    # Cross-covariance (model→scene)
    H = M0 @ S0.T                                   # 3x3

    # SVD and proper rotation
    U, _, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T
    if np.linalg.det(R) < 0:                        # fix improper rotation
        Vt[-1, :] *= -1
        R = Vt.T @ U.T

    # Translation: p = c_s - R c_m
    p = (c_s - R @ c_m).reshape(3)

    # Drake transform
    X_BA = RigidTransform(RotationMatrix(R), p)
    return X_BA

## Part 2: Point correspondence from closest point
The ```least_squares_transform``` function assumes that the point correspondence is known. Unfortunately, this is often not the case, so we will have to estimate the point correspondence as well. A common heuristics for estimating point correspondence is the closest point/nearest neighbor.

We have implemented the closest neighbors using [scipy's implementation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html) of [k-d trees](https://en.wikipedia.org/wiki/K-d_tree).

In [10]:
def nearest_neighbors(
    scene: np.ndarray, model: np.ndarray
) -> tuple[np.ndarray, np.ndarray]:
    """
    Find the nearest (Euclidean) neighbor in model for each
    point in scene
    Args:
        scene: 3xN numpy array of points
        model: 3xM numpy array of points
    Returns:
        distances: (N, ) numpy array of Euclidean distances from each point in
            scene to its nearest neighbor in model.
        indices: (N, ) numpy array of the indices in model of each
            scene point's nearest neighbor - these are the c_i's
    """
    kdtree = KDTree(model.T)

    distances, indices = kdtree.query(scene.T, k=1)

    return distances.flatten(), indices.flatten()

## Part 3: Iterative Closest Point (ICP)
Now you should be able to register two point clouds iteratively by first finding/updating the estimate of point correspondence with ```nearest_neighbors``` and then computing the transform using ```least_squares_transform```. You may refer to the explanation from [textbook](http://manipulation.csail.mit.edu/pose.html#icp).

**YOUR TASK**: In the cell below, complete the implementation of ICP algorithm using the  ```nearest_neighbors``` and ```least_squares_transform``` methods from above.

In [ ]:
def icp(
    scene: np.ndarray,
    model: np.ndarray,
    max_iterations: int = 20,
    tolerance: float = 1e-3,
) -> tuple[RigidTransform, float, int]:
    """
    Perform ICP to return the correct relative transform between two set of points.
    Args:
        scene: 3xN numpy array of points
        model: 3xM numpy array of points
        max_iterations: max amount of iterations the algorithm can perform.
        tolerance: tolerance before the algorithm converges.
    Returns:
      X_BA: A RigidTransform object that maps point_cloud_A on to point_cloud_B
            such that
                        X_BA.multiply(model) ~= scene,
      mean_error: Mean of all pairwise distances.
      num_iters: Number of iterations it took the ICP to converge.
    """
    X_BA = RigidTransform()

    mean_error = 0
    num_iters = 0
    prev_error = np.inf  # Initialize to infinity so first iteration runs

    while True:
        num_iters += 1

        # Step 1: Transform model points using current transform
        model_transformed = X_BA.multiply(model)  # 3xM

        # Step 2: Find nearest neighbors efficiently
        # Compute pairwise squared distances between scene and model_transformed
        scene_sq = np.sum(scene**2, axis=0, keepdims=True).T  # Nx1
        model_sq = np.sum(model_transformed**2, axis=0, keepdims=True)  # 1xM
        cross_term = scene.T @ model_transformed  # NxM

        dists_sq = scene_sq + model_sq - 2 * cross_term  # NxM

        # For each model point, find index of closest scene point
        closest_indices = np.argmin(dists_sq, axis=0)  # M indices

        # Step 3: Get corresponding scene points
        scene_matched = scene[:, closest_indices]  # 3xM

        # Step 4: Compute mean error
        distances = np.sqrt(np.sum((scene_matched - model_transformed)**2, axis=0))
        mean_error = np.mean(distances)

        # Step 5: Compute incremental transform
        X_increment = least_squares_transform(scene_matched, model_transformed)

        # Step 6: Update cumulative transform
        X_BA = X_increment.multiply(X_BA)

        #############

        if abs(mean_error - prev_error) < tolerance or num_iters >= max_iterations:
            break

        prev_error = mean_error

        meshcat.SetTransform("red_bunny", X_BA)

    return X_BA, mean_error, num_iters

Now you should be able to visualize the registration of the Stanford bunny! Have fun!

In [18]:
icp(pointcloud_scene, pointcloud_model, max_iterations=30, tolerance=1e-5)

(RigidTransform(
   R=RotationMatrix([
     [0.9999999999999998, -5.551115123125783e-17, 1.1102230246251565e-16],
     [1.6306400674181987e-16, 0.8660254037844379, -0.4999999999999995],
     [-1.6653345369377348e-16, 0.49999999999999967, 0.8660254037844383],
   ]),
   p=[-0.09999999999999987, 0.10000000000000006, 0.10000000000000053],
 ),
 np.float64(5.43071954799835e-16),
 28)

You may use the grader below to check your implementations

In [21]:
Grader.grade_output([TestICP], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 6/6.

Score for Test icp implementation is 3/3.

Score for Test least square transform is 3/3.


## VERIFICATION IN GRADESCOPE

**Prerequisites:** You must complete ALL the TODOs above before these verification exercises will work!

**Instructions:** Complete the exercises that follow and upload your solutions to Gradescope


## Verification 1: Least Squares Transforms

**Question:** What does the `X_BA` rotation matrix returned from ICP above correspond to? Select the appropraite answer in Gradescope.

A. A +60° rotation about the z-axis 
B. A +30° rotation about the y-axis 
C. A -30° rotation about the x-axis 
D. A +30° rotation about the x-axis

**HINT**: Use the `RotationMatrix.MakeX/Y/ZRotation()` methods to create rotation matrices. 